In [1]:
import numpy as np
import pandas as pd

# Создание новых признаков

In [2]:
path = 'E:/retailhero-uplift/data'

In [5]:
uplift = pd.read_csv(path + '/train.csv', encoding='utf-8')
uplift.head()

,client_id,first_issue_date,first_redeem_date,age,gender,treatment_flg,target
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,0,1
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,1,1
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,1,1
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F,1,1
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U,1,1


In [10]:
uplift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200039 entries, 0 to 200038
Data columns (total 7 columns):
client_id            200039 non-null object
first_issue_date     200039 non-null object
first_redeem_date    182493 non-null object
age                  200039 non-null int64
gender               200039 non-null object
treatment_flg        200039 non-null int64
target               200039 non-null int64
dtypes: int64(3), object(4)
memory usage: 10.7+ MB


In [6]:
purchases = pd.read_csv(path + '/purchases.csv', encoding='utf-8')
purchases.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [11]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45786568 entries, 0 to 45786567
Data columns (total 13 columns):
client_id                  object
transaction_id             object
transaction_datetime       object
regular_points_received    float64
express_points_received    float64
regular_points_spent       float64
express_points_spent       float64
purchase_sum               float64
store_id                   object
product_id                 object
product_quantity           float64
trn_sum_from_iss           float64
trn_sum_from_red           float64
dtypes: float64(8), object(5)
memory usage: 4.4+ GB


Извлечение признаков из временных меток:
 - месяц
 - день недели
 - час
 - разница между first_issue_date и first_redeem_date

In [9]:
def feature_extraction(df1, df2):
    df1['first_issue_date_month'] = df1['first_issue_date'].dt.month
    df1['first_redeem_date_month'] = df1['first_redeem_date'].dt.month
    df1['first_issue_date_weekday'] = df1['first_issue_date'].dt.weekday
    df1['first_redeem_date_weekday'] = df1['first_redeem_date'].dt.weekday
    df1['first_issue_date_hour'] = df1['first_issue_date'].dt.hour
    df1['first_redeem_date_hour'] = df1['first_redeem_date'].dt.hour
    df1['reedem_issue_diff'] = df1['first_redeem_date'] - df['first_issue_date']
    df2['transaction_datetime_weekday'] = df2['transaction_datetime'].dt.weekday
    df2['transaction_datetime_hour'] = df2['transaction_datetime'].dt.hour

Из файла с покупками сделаем такие признаки за всю историю и последние 30 дней: 
   - общее количество покупок(штуки и суммы)
   - количество накопленных баллов
   - количество потраченных баллов
   - баланс баллов
   - количество уникальных магазинов, где были совершены покупки
   - количество дней между первой и последней покупкой

In [12]:
def purchase_features(df):
    last_cols = ['regular_points_received', 'express_points_received','regular_points_spent', 
                 'express_points_spent', 'purchase_sum','store_id', 'product_quantity']
    all_hist = df.groupby(['client_id','transaction_id'])[last_cols].last()
    last_month = df[df['transaction_datetime'] > '2019-02-18'].groupby(['client_id','transaction_id'])[last_cols].last()
    
    df['total_purchase_sum'] = all_hist.groupby('client_id')['purchase_sum'].sum()
    df['last_30_purchase_sum'] = last_month.groupby('client_id')['purchase_sum'].sum()
    df['total_product_quantity'] = all_hist.groupby('client_id')['product_quantity'].sum()
    df['last_30_product_quantity_sum'] = last_month.groupby('client_id')['product_quantity'].sum()
    df['total_regular_points_received'] = all_hist.groupby('client_id')['regular_points_received'].sum()
    df['last_30_regular_points_received'] = last_month.groupby('client_id')['regular_points_received'].sum()
    df['total_express_points_received'] = all_hist.groupby('client_id')['express_points_received'].sum()
    df['last_30_express_points_received'] = last_month.groupby('client_id')['express_points_received'].sum()
    df['total_regular_points_spent'] = all_hist.groupby('client_id')['regular_points_spent'].sum()
    df['last_30_regular_points_spent'] = last_month.groupby('client_id')['regular_points_spent'].sum()
    df['total_express_points_spent'] = all_hist.groupby('client_id')['express_points_spent'].sum()
    df['last_30_express_points_spent'] = last_month.groupby('client_id')['express_points_spent'].sum()
    df['total_regular_points_balance'] = df['total_regular_points_received'] - df['total_regular_points_spent']
    df['total_express_points_balance'] = df['total_express_points_received'] - df['total_express_points_spent']
    df['last_30_regular_points_balance'] = df['last_30_regular_points_received'] - df['last_30_regular_points_spent']
    df['last_30_express_points_balance'] = df['last_30_express_points_received'] - df['total_express_points_spent']
    df['unique_stores'] = all_hist.groupby('client_id')[['store_id']].nunique()
    df['last_30_unique_stores'] = last_month.groupby('client_id')[['store_id']].nunique()
    df['days_delay'] = (all_hist.groupby('client_id')['transaction_datetime'].iloc[0] - 
                        all_hist.groupby('client_id')['transaction_datetime'].iloc[-1]).dt.days
    df['last_days_delay'] = (last_month.groupby('client_id')['transaction_datetime'].iloc[0] - 
                        last_month.groupby('client_id')['transaction_datetime'].iloc[-1]).dt.days

Добавим описательные статистики:
 - среднее
 - стандартное отклонение
 - квартили
 - минимумы
 - максимумы

И добавим функциональные зависимости:
 - полиномы
 - логарифмы
 - экспоненты